#NLP BERT Classification

BERT (Bidirectional Encoder Representations from Transformers) is a Natural Language Processing Model proposed by researchers at Google Research in 2018. When it was proposed it achieve state-of-the-art accuracy on many NLP and NLU tasks

in this program bert base model is used and for loading the model pytorch lib is used.

In [32]:
!pip install pytorch_pretrained_bert pytorch-nlp

     |████████████████████████████████| 133kB 16.2MB/s 
     |████████████████████████████████| 92kB 12.2MB/s 
     |████████████████████████████████| 133kB 55.0MB/s 
     |████████████████████████████████| 7.3MB 18.6MB/s 
     |████████████████████████████████| 71kB 12.0MB/s 
ERROR: botocore 1.20.22 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


#Importing Libs

In [33]:
import sys
import numpy as np
import pandas as pd
import random as rn
import torch
from pytorch_pretrained_bert import BertModel
from torch import nn
from pytorch_pretrained_bert import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output

## Process the files

Since, we have the system call names for only one file, we will be using that to convert other files' system call number to system call name

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Following file is having the system call names
intrusion_1_txt = pd.read_csv('/content/drive/MyDrive/NLP_BERT/result-sm-10763.txt',header=None,sep="\s+",names=["users","syscalls"])

In [4]:
# Following file is having the system call ids for the same above
intrusion_1_i = pd.read_csv('/content/drive/MyDrive/NLP_BERT/sm-10763.int',header=None,sep="\s+",names=["users","syscalls"])

In [5]:
# We will use this two file to map system call number and call names
intrusion_1_i["text_syscalls"] = intrusion_1_txt.syscalls

In [6]:
# Create Map of Sys calls to  int to text
call_maps = {}
for i in range(len(intrusion_1_i.syscalls)):
    call_maps[intrusion_1_i.syscalls[i]] = intrusion_1_i.text_syscalls[i]

In [7]:
# Following is the System number and System name map and will use to convert other 
call_maps

{2: 'read',
 3: 'write',
 4: 'open',
 5: 'close',
 7: 'creat',
 8: 'link',
 9: 'unlink',
 11: 'chdir',
 14: 'chmod',
 17: 'stat',
 18: 'lseek',
 19: 'getpid',
 23: 'getuid',
 27: 'fstat',
 32: 'access',
 45: 'getgid',
 50: 'ioctl',
 56: 'umask',
 59: 'getpagesize',
 66: 'mmap',
 78: 'setitimer',
 83: 'gethostname',
 85: 'getdtablesize',
 88: 'fcntl',
 89: 'select',
 93: 'socket',
 94: 'connect',
 100: 'bind',
 104: 'sigvec',
 105: 'sigblock',
 106: 'sigsetmask',
 108: 'sigstack',
 112: 'gettimeofday',
 121: 'recvfrom',
 128: 'sendto',
 138: 'getrlimit',
 155: 'getdomainname',
 167: 'getdents'}

### Convert other system call numbers to names

#### Convert call of Intrusion 2 

In [8]:

intrusion_2 = pd.read_csv("/content/drive/MyDrive/NLP_BERT/sm-10801.int",header=None,sep="\s+",names=["users","syscalls"])

intrusion_2["text_syscalls"] = "a"  # Tmp place holder

for i in range(len(intrusion_2.syscalls)):
    
    if intrusion_2.syscalls[i] in call_maps.keys():
        # If there is system call name for the number in dict then use that
        intrusion_2.text_syscalls[i] = call_maps[intrusion_2.syscalls[i]]
    else:
        # If there is not system call name for the nuber in the dict then randomly select one and 
        # Sotre it 
        scall =np.random.choice(list(call_maps.values()))
        intrusion_2.text_syscalls[i] = scall

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [9]:
# Convert call of Bounce -1  
bounce_1 = pd.read_csv("/content/drive/MyDrive/NLP_BERT/bounce-1.int",header=None,sep="\s+",names=["users","syscalls"])

bounce_1 ["text_syscalls"] = "a"  # Tmp place holder

for i in range(len(bounce_1.syscalls)):
    
    if bounce_1.syscalls[i] in call_maps.keys():
        # If there is system call name for the number in dict then use that
        bounce_1.text_syscalls[i] = call_maps[bounce_1.syscalls[i]]
    else:
        # If there is not system call name for the nuber in the dict then randomly select one and 
        # Sotre it 
        scall =np.random.choice(list(call_maps.values()))
        bounce_1.text_syscalls[i] = scall

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [10]:
# Convert call of Bounce -1  
bounce_2 = pd.read_csv("/content/drive/MyDrive/NLP_BERT/bounce-2.int",header=None,sep="\s+",names=["users","syscalls"])

bounce_2 ["text_syscalls"] = "a"  # Tmp place holder

for i in range(len(bounce_2.syscalls)):
    
    if bounce_2.syscalls[i] in call_maps.keys():
        # If there is system call name for the number in dict then use that
        bounce_2.text_syscalls[i] = call_maps[bounce_2.syscalls[i]]
    else:
        # If there is not system call name for the nuber in the dict then randomly select one and 
        # Sotre it 
        scall =np.random.choice(list(call_maps.values()))
        bounce_2.text_syscalls[i] = scall

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


## Combine The data

In [11]:
# Combine individual
intruder = intrusion_1_i.append(intrusion_2)  # Combine Intrution data
user_data = bounce_1.append(bounce_2)  # Normal User Data

In [12]:
# Drop Neumeric data line
intruder = intruder.drop("syscalls",axis=1)
user_data = user_data.drop("syscalls",axis=1)

In [13]:
intruder

,users,text_syscalls
0,10765,open
1,10765,read
2,10765,mmap
3,10765,mmap
4,10765,open
...,...,...
368,10803,close
369,10803,close
370,10803,close
371,10803,close


In [14]:
user_data

,users,text_syscalls
0,1401,open
1,1401,read
2,1401,mmap
3,1401,mmap
4,1401,open
...,...,...
734,1414,close
735,1414,close
736,1414,close
737,1414,close


In [15]:
df_intr = intruder.rename(columns= {"users":"PID", "text_syscalls":"Syscall"})

In [16]:
df_usr = user_data.rename(columns= {"users":"PID", "text_syscalls":"Syscall"})

In [17]:
df_intr.head()

,PID,Syscall
0,10765,open
1,10765,read
2,10765,mmap
3,10765,mmap
4,10765,open


In [18]:
df_usr.head()

,PID,Syscall
0,1401,open
1,1401,read
2,1401,mmap
3,1401,mmap
4,1401,open


In [19]:
# Group normal df by PID, so the sequences appear by PID instead of by order cuz it can be interrupted by other PID
filt = df_usr.groupby(['PID'])
normal = pd.DataFrame()

pidList = df_usr['PID'].unique()   # Get unique PIDs in pid
for p in pidList:
    normal = normal.append(filt.get_group(p), ignore_index=True)
# Drop PID column to train model on just syscall
normal.drop(axis = 1, columns='PID', inplace= True)


# Do the same thing to intrusion PID
filt = df_intr.groupby(['PID'])
intrusion = pd.DataFrame()

pidList = df_intr['PID'].unique()   # Get unique PIDs in pid
for p in pidList:
    intrusion = intrusion.append(filt.get_group(p), ignore_index=True)
intrusion.drop(axis = 1, columns='PID', inplace=True)

In [20]:
normal.head(10)

,Syscall
0,open
1,read
2,mmap
3,mmap
4,open
5,getrlimit
6,mmap
7,close
8,getuid
9,getgid


In [21]:
intrusion.head(10)

,Syscall
0,open
1,read
2,mmap
3,mmap
4,open
5,getrlimit
6,mmap
7,close
8,getuid
9,getgid


In [22]:
# Function dropRows calc number of rows that need to be dropped so that the df can be reshape (-1,4) evenly
def dropRows(df):
    r, c = df.shape
    extra = r % 4
    print('Drop', extra ,'rows')
    while extra != 0:
        df.drop([r-1], axis=0, inplace=True)
        r, c= df.shape
        extra = r%4

In [23]:
# Drop the 2 last rows in normal, so it's divisible by 4 (normal row = 818)
# Drop 1 last rows in intrusion, so it's divisible by 4 (intrusion row = 373)
print('Normal shape:',normal.shape)
dropRows(normal)
print('Normal new shape:',normal.shape)

print('Intrusion shape:',intrusion.shape)
dropRows(intrusion)
print('Intrusion new shape:',intrusion.shape)

Normal shape: (1032, 1)
Drop 0 rows
Normal new shape: (1032, 1)
Intrusion shape: (746, 1)
Drop 2 rows
Intrusion new shape: (744, 1)


In [24]:
# Reshape data from 1 row multiple columns to multiple rows with 4 cols
arr = normal.to_numpy()
type(arr)
arr = arr.reshape((-1, 4))
print('normal new shape:',arr.shape)
#Change numpy arr into df
normal = pd.DataFrame(arr)


# Reshape data from 1 row multiple columns to multiple rows with 4 cols
arr = intrusion.to_numpy()
type(arr)
arr = arr.reshape((-1, 4))
print('intrusion new shape:',arr.shape)
#Change numpy arr into df
intrusion = pd.DataFrame(arr)

normal new shape: (258, 4)
intrusion new shape: (186, 4)


In [25]:
normal.head()

,0,1,2,3
0,open,read,mmap,mmap
1,open,getrlimit,mmap,close
2,getuid,getgid,open,fstat
3,mmap,close,open,read
4,mmap,mmap,close,open


In [26]:
intrusion.head()

,0,1,2,3
0,open,read,mmap,mmap
1,open,getrlimit,mmap,close
2,getuid,getgid,open,fstat
3,mmap,close,open,read
4,mmap,mmap,close,close


In [27]:
normal_txt = []
normal_label = []

intrusion_txt = []
intrusion_label = []


# Connect all calls names with space in between 
for i in range(normal.shape[0]):
    normal_txt.append(" ".join(np.array(normal.iloc[i,:]).astype(str)))
    normal_label.append("normal")


for i in range(intrusion.shape[0]):
    intrusion_txt.append(" ".join(np.array(intrusion.iloc[i,:]).astype(str)))
    intrusion_label.append("intrusion")

In [28]:
All_data = normal_txt + intrusion_txt  # Combine all data set
All_labels = normal_label + intrusion_label

In [29]:
All_data = np.array(All_data) #Convert list dataset into np array x and y both
All_labels = np.array(All_labels)

Randomly shuffel all the data

In [30]:
sd = 1114
rnd = np.random.RandomState(sd)
rnd.shuffle(All_data)

sd = 1114
rnd = np.random.RandomState(sd)
rnd.shuffle(All_labels)

###Using sklearn train test split model to split the train and test data

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(All_data, All_labels, test_size=0.2, random_state=1)

In [34]:
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)

Store the train and test data set as per Setting of IMDB dataset Dictionary settings

In [63]:
train_data = []
test_data = []
for i,j in zip(X_train,y_train):
    tmp_dict = {'ids':j,'text':i}
    train_data.append(tmp_dict)
for i,j in zip(X_test,y_test):
    tmp_dict = {'ids':j,'text':i}
    test_data.append(tmp_dict)

Store the train test preperd dictionary dataset in to tuples

In [35]:
train_texts = tuple(X_train.tolist())
train_labels = tuple(y_train.tolist())

test_texts = tuple(X_test.tolist())
test_labels = tuple(y_test.tolist())


len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(355, 355, 89, 89)

## Prepare the Data

call the bert tokenizer function and set the tokens to the sentances 

In [36]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 7607450.10B/s]


In [39]:
tokenizer.tokenize(X_train[8])

['close', 'close', 'close', 'open']

In [40]:
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], test_texts))

len(train_tokens), len(test_tokens)                   
                   

(355, 89)

In [41]:
train_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, train_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, test_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")

train_tokens_ids.shape, test_tokens_ids.shape

((355, 512), (89, 512))

In [42]:
y_test #Chaeck the test actual results

array(['normal', 'normal', 'normal', 'normal', 'normal', 'normal',
       'normal', 'normal', 'intrusion', 'normal', 'normal', 'normal',
       'normal', 'normal', 'normal', 'intrusion', 'intrusion', 'normal',
       'normal', 'intrusion', 'normal', 'intrusion', 'normal',
       'intrusion', 'intrusion', 'normal', 'normal', 'normal',
       'intrusion', 'normal', 'normal', 'intrusion', 'intrusion',
       'normal', 'intrusion', 'normal', 'normal', 'normal', 'normal',
       'normal', 'normal', 'normal', 'normal', 'intrusion', 'intrusion',
       'normal', 'normal', 'intrusion', 'intrusion', 'intrusion',
       'normal', 'normal', 'normal', 'normal', 'intrusion', 'intrusion',
       'normal', 'normal', 'normal', 'normal', 'normal', 'intrusion',
       'normal', 'normal', 'normal', 'normal', 'normal', 'intrusion',
       'normal', 'normal', 'intrusion', 'normal', 'normal', 'normal',
       'intrusion', 'intrusion', 'normal', 'normal', 'intrusion',
       'normal', 'normal', 'intrusion', 

In [43]:
train_y = np.array(train_labels) == 'normal'
test_y = np.array(test_labels) == 'normal'
train_y.shape, test_y.shape, np.mean(train_y), np.mean(test_y)

((355,), (89,), 0.5521126760563381, 0.6966292134831461)

add bert masks to the train and test models

In [44]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]

# Bert Model

Importing the Bert classifer Trained model in program and we can also load other bert models form github link
"https://github.com/google-research/bert"

In [49]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba
        

For this model GPU is very usefull with out GPU it take a lot of time to run so it is recomende to run on GPU if not avilable then you can also use cpu but it is better to used cloud computing like goolge colab

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

store the bert classifer model in to new variable for training 

In [52]:
bert_clf = BertBinaryClassifier()
bert_clf = bert_clf.cuda()


100%|██████████| 407873900/407873900 [00:06<00:00, 66705546.83B/s]


In [53]:
x = torch.tensor(train_tokens_ids[:3]).to(device)
y, pooled = bert_clf.bert(x, output_all_encoded_layers=False)
x.shape, y.shape, pooled.shape

(torch.Size([3, 512]), torch.Size([3, 512, 768]), torch.Size([3, 768]))

In [54]:
y = bert_clf(x)
y.cpu().detach().numpy()

array([[0.40634346],
       [0.43269828],
       [0.43176568]], dtype=float32)

In [55]:
y, x, pooled = None, None, None
torch.cuda.empty_cache()
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'439.065088M'

# Fine-tune BERT

Select the batch size and number of epoch for training.
now in this case data set is low so it is better to use less epoch becouse bert base model has 12 layers and 768 peramerts

In [56]:
BATCH_SIZE = 4
EPOCHS = 10

Load the traing and testing dataset in to model

In [57]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'439.065088M'

In [58]:
train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)


In [59]:
param_optimizer = list(bert_clf.sigmoid.named_parameters()) 
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

Bert model is used adam optimzer for more detail on the model use the github link given above

In [60]:
optimizer = Adam(bert_clf.parameters(), lr=3e-6)

In [61]:
 torch.cuda.empty_cache()

###Compile the bert classifer model

In [64]:
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
        logits = bert_clf(token_ids, masks)
        
        loss_func = nn.BCELoss()

        batch_loss = loss_func(logits, labels)
        train_loss += batch_loss.item()
        
        
        bert_clf.zero_grad()
        batch_loss.backward()
        

        clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        optimizer.step()
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))
        

Epoch:  10
88/88.75 loss: 0.5975740893503253 


#Test the train model with tesing dataset and Measure the accuracy of the model

In [65]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
    

In [66]:
np.mean(bert_predicted)

0.6179775280898876

In [67]:
print(classification_report(test_y, bert_predicted))

              precision    recall  f1-score   support

       False       0.35      0.44      0.39        27
        True       0.73      0.65      0.68        62

    accuracy                           0.58        89
   macro avg       0.54      0.54      0.54        89
weighted avg       0.61      0.58      0.60        89

